In [11]:
import logging
import os
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from gensim import corpora, models, similarities

In [13]:
if (os.path.exists("/tmp/deerwester.dict")):
    dictionary = corpora.Dictionary.load('/tmp/deerwester.dict')
    corpus = corpora.MmCorpus('/tmp/deerwester.mm')
    print("Used files generated from first tutorial")
else:
    print("Please run first tutorial to generate data set")

2017-02-06 14:50:57,725 : INFO : loading Dictionary object from /tmp/deerwester.dict
2017-02-06 14:50:57,726 : INFO : loaded /tmp/deerwester.dict
2017-02-06 14:50:57,728 : INFO : loaded corpus index from /tmp/deerwester.mm.index
2017-02-06 14:50:57,729 : INFO : initializing corpus reader from /tmp/deerwester.mm
2017-02-06 14:50:57,730 : INFO : accepted corpus with 9 documents, 12 features, 28 non-zero entries


Used files generated from first tutorial
